In [16]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from qiskit import Aer, ClassicalRegister, QuantumRegister, QuantumCircuit, execute
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.visualization import plot_histogram
from PIL import Image
%matplotlib inline

# Replace token with your token and set the backend service to quantumComputer
# quantumComputer = QiskitRuntimeService(channel="ibm_quantum", token="69d212ec7abdb18dfc6b789b7a3fdf38b523ed311a1acd45f6bc2e9350fdcd2062f7f2ad5f334902f2deae27c8b8c51d0231a795c39f2f085444b2f7a78f53b8").least_busy(operational=True, simulator=False)
simulator = Aer.get_backend('qasm_simulator')

# Start specific game
def startGame(gameName):
    if gameName == "Fortune Teller":
        print(f"Starting {gameName} 🔮!")
        fateGame()
        print()
    elif gameName == "Beat the Flip":
        print(f"Starting {gameName} 🪙!")
        flipGame()
        print()
    elif gameName == "Number Guess":
        print(f"Starting {gameName}!")
        guessGame()
        print()
    elif gameName == "Quantum Art":
        print(f"Starting {gameName} 🖼️!")
        quantumArt()
        print()
    else:
        print("Restart Please")

# Quantum Computing Fate Game uses randomness based on quantum uncertainity 
def fateGame():
    # Create Quantum Circuit with 3 quantum register and 3 classical registers 
    qr = QuantumRegister(3)
    cr = ClassicalRegister(3)
    circuit = QuantumCircuit(qr, cr)
    
    # Use hadmard gate to set supererposition and measure
    circuit.h(qr)
    circuit.measure(qr, cr)

    # Execute circuit and store results
    print("Think about a yes or no question 🧞")
    result = execute(circuit, backend = simulator).result().get_counts(circuit)

    # Response based on the measurements
    response = list(result.keys())[-1]
        
    # Decode the response
    if response == "000":
        print("It is certain!")
    elif response == "001":
        print("Without any question.")
    elif response == "010":
        print("Yes")
    elif response == "100":
        print("Maybe?")
    elif response == "101":
        print("Forget about it!")
    elif response == "110":
        print("I wouldn't count on it.")
    elif response == "011":
        print("No")
    elif response == "111":
        print("Think harder...")
    else:
        print("Hmmm. I can't figure this one out.")

# Quantum Computing Flip Game
def flipGame():
    # Create UI with play button, dropdown, and amount of shots field
    playButton = widgets.Button(description = "Play")
    dropdown = widgets.Dropdown(options = [("Nothing"), ("Flip"), ("Entangle")], description = "Choose: ")
    userShotsInput = widgets.IntText(value = 1, description = "Shots: ")
    output = widgets.Output()

    # Execute when button clicked
    def buttonClicked(i):
        with output:
            try:
                # Get shors from user, check validity
                userShots = int(userShotsInput.value)
                if userShots < 1:
                    raise ValueError("Number of shots should be 1 or more.")
            except ValueError as e:
                print(f"Invalid input: {e}")
                return
            
            # Create Quantum Circuit with 2 quantum register and 2 classical registers 
            qr = QuantumRegister(2)
            cr = ClassicalRegister(2)
            circuit = QuantumCircuit(qr, cr)

            # Use hadmard gate to set first qubit to supererposition
            circuit.h(qr[0])

            # If nothing use identity gate, if flip use Pauli-X, if entangle use hadmark and then a controlled not
            if dropdown.value == "Nothing":
                circuit.id(qr[0])
            elif dropdown.value == "Flip":
                circuit.x(qr[0])
            elif dropdown.value == "Entangle":
                circuit.h(qr[1])
                circuit.cx(qr[1], qr[0])

            # Use hadmard gate to set first qubit back into supererposition
            circuit.h(qr[0])
            circuit.measure(qr, cr)

            # Execute circuit and store results
            result = execute(circuit, backend = simulator, shots = userShots).result().get_counts()

            # Use the possible states of the qubits to determine winner
            if len(result) == 1 and list(result.keys())[0] == '00':
                print("Quantum Computer Won!")
            elif len(result) == 1 and list(result.keys())[0] == '11':
                print("You Won!")
            elif len(result) == 2 and result["00"] > result["10"]:
                print("Quantum Computer Won! That Was Close...")
            elif len(result) == 2 and result["00"] < result["10"]:
                print("You Won! That Was Close...")
            else:
                print("Either Quantum or You Wins")
    
    # Show components and handle button
    playButton.on_click(buttonClicked)
    display(widgets.VBox([dropdown, userShotsInput, playButton, output]))
 
# Quantum Computing Guess Game based on Bernstein-Vazirani algorithm
def guessGame():
    # Get input from user
    userInput = widgets.IntText(value = 1, description = "Enter a number: ")
    guessButton = widgets.Button(description = "Guess my Number")
    output = widgets.Output()
    
    # Execute when button clicked
    def on_button_clicked(b):
        with output:
            try:
                # Convert input to binary
                binaryValue = bin(userInput.value)[2:]
                binaryValueLength = len(binaryValue)
                
                # Create a circuit with 1 more quantum gate than the binary length
                circuit = QuantumCircuit(binaryValueLength+1, binaryValueLength)

                # Use hadmard gate to set last qubit in supererposition, then Z gate
                circuit.h(binaryValueLength)
                circuit.z(binaryValueLength)
                
                # Use hadmard gate to set everything but last qubit in supererposition
                for i in range(binaryValueLength):
                    circuit.h(i)

                # Reverse binary value, do CNOT gate if binary value is 1 of all but last qubit
                binaryValue = binaryValue[::-1]
                for i in range(binaryValueLength):
                    if binaryValue[i] == "1":
                        circuit.cx(i, binaryValueLength)

                # Use hadmard gate for all except last qubits
                for i in range(binaryValueLength):
                    circuit.h(i)

                # Measure all but last qubits
                for i in range(binaryValueLength):
                    circuit.measure(i, i)

                # Execute circuit and convert back to decimal
                result = execute(circuit, backend = simulator).result().get_counts()
                resultDecimal = int(list(result.keys())[0], 2)
                
                # Show user results and the circuit to get results
                print(f"Your number is {str(resultDecimal)} based on the quantum circuit below")
                print(circuit)
            except Exception as e:
                print(f"Error try again: {e}")

    # Show components and handle button
    guessButton.on_click(on_button_clicked)
    display(widgets.VBox([userInput, guessButton, output]))
 
# Quantum Art Drawing
def quantumArt():
    # Get input from user
    userInputQubits = widgets.IntText(value = 2, description = "Qubits to Use: ")
    userInputMultiplier = widgets.IntText(value = 1, description = "Multiplier to Use: ")
    drawButton = widgets.Button(description = "Draw Quantum Art")
    output = widgets.Output()

    # Execute when button clicked
    def buttonClicked(b):
        with output:
            # Ensure qubit input is greater than 2 and multiplier input is greater than 1
            try:
                numberOfQubits = int(userInputQubits.value)
                multiplier = int(userInputMultiplier.value)
                
                if numberOfQubits < 2:
                    raise ValueError("Number of qubits should be 2 or more.")
                    
                if multiplier < 1:
                    raise ValueError("Multiplier should be 1 or more")
                
                draw(numberOfQubits, multiplier)
            except ValueError as e:
                print(f"Invalid input: {e}")
                return

    # Draw a random circuit based on the number of qubits needed
    def draw(numberOfQubits, multiplier):
        # Create Quantum Circuit with specified quantum register and corresponding classical registers 
        circuit = QuantumCircuit(numberOfQubits, numberOfQubits)
        
        # Create list to store results
        results = []
        
        # Go through the qubits specified times a multiplier to see how many random gates per operation
        for _ in range(numberOfQubits * multiplier):
            # Randomly chose a gate and select a qubit without replacing
            randomGate = np.random.choice(['h', 'x', 'y', 'z', 'cx', 'cz', 'swap'])
            qubitIndex = np.random.choice(numberOfQubits, size = circuit.num_qubits, replace = False)
             
            # Apply random gate to the selected qubit
            if randomGate == 'h':
                circuit.h(qubitIndex)
            elif randomGate == 'x':
                circuit.x(qubitIndex)
            elif randomGate == 'y':
                circuit.y(qubitIndex)
            elif randomGate == 'z':
                circuit.z(qubitIndex)
            elif randomGate == 'cx':
                control, target = np.random.choice(numberOfQubits, size=2, replace=False)
                circuit.cx(control, target)
            elif randomGate == 'cz':
                control, target = np.random.choice(numberOfQubits, size=2, replace=False)
                circuit.cz(control, target)
            elif randomGate == 'swap':
                qubit1, qubit2 = np.random.choice(numberOfQubits, size=2, replace=False)
                circuit.swap(qubit1, qubit2)
            
            # Measure each qubit after applying the random gate
            for i in range(numberOfQubits):
                circuit.measure(i, i)
                result = execute(circuit, backend = simulator, memory = True).result()
                memory = result.get_memory()
                
                # Measurements for each shot
                for x in range(result.results[0].shots):
                    results.append(memory[x])
        
        # Execute the final circuit
        result = execute(circuit, backend = simulator).result()
        plot_histogram(result.get_counts(), ax=plt.gca())
        plt.show()
        
        # Combine results into binary string and save to image
        combinedResults = "".join(results)
        binaryToImage(combinedResults, 1000, 400).save("generative-art.jpg")
        
        # Print the final circuit
        print(circuit)

    # Turn binary into image
    def binaryToImage(input, width, height):
        # Create image with specified width and height in black and white
        image = Image.new("1", (width, height), color = 1)
        pixels = image.load()

        # Iterate through input and set pixel values accordingly
        index = 0
        for i in range(height):
            for j in range(width):
                pixel_value = int(input[index])
                pixels[j, i] = pixel_value
                
                index += 1
                
                if index == len(input):
                    break
            if index == len(input):
                break
        return image
    
    # Show components and handle button
    drawButton.on_click(buttonClicked)
    display(widgets.VBox([userInputQubits, userInputMultiplier, drawButton, output]))
        
# Handle button click
def buttonClicked(b):
    gameName = b.description
    startGame(gameName)

# Buttons for games and event handling
fateButton = widgets.Button(description="Fortune Teller")
flipButton = widgets.Button(description="Beat the Flip")
numberButton = widgets.Button(description="Number Guess")
artButton = widgets.Button(description="Quantum Art")

fateButton.on_click(buttonClicked)
flipButton.on_click(buttonClicked)
numberButton.on_click(buttonClicked)
artButton.on_click(buttonClicked)

# Show buttons
display(fateButton)
display(flipButton)    
display(numberButton)
display(artButton)

Button(description='Fortune Teller', style=ButtonStyle())

Button(description='Beat the Flip', style=ButtonStyle())

Button(description='Number Guess', style=ButtonStyle())

Button(description='Quantum Art', style=ButtonStyle())